In [1]:
# Suppress common warnings and configure environment
import os
import warnings
import logging

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TF warnings
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN if causing issues
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Suppress tokenizer warnings

# Set logging levels
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print("Environment configured - warnings suppressed!")

Environment configured - warnings suppressed!


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# env file
from dotenv import load_dotenv
import os
load_dotenv()

True

## Download Dataset

In [3]:
from huggingface_hub import HfApi

# Initialize the Hugging Face API
api = HfApi()
user_info = api.whoami(token=os.getenv("HF_TOKEN"))
print(user_info)

/home/alibina/repo/Training_LLM/chemLLM/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'type': 'user', 'id': '641e99912bb8b89e81d099b3', 'name': 'iAli61', 'fullname': 'Ali Bina', 'isPro': False, 'avatarUrl': '/avatars/dff6c063bbc2be75d628727692ff4092.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'HF_TOKEN', 'role': 'fineGrained', 'createdAt': '2025-06-29T15:29:08.821Z', 'fineGrained': {'canReadGatedRepos': True, 'global': [], 'scoped': [{'entity': {'_id': '641e99912bb8b89e81d099b3', 'type': 'user', 'name': 'iAli61'}, 'permissions': ['repo.content.read', 'repo.write', 'user.webhooks.read', 'user.webhooks.write', 'collection.read', 'collection.write']}]}}}}


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")

2025-07-05 14:50:25.447519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751719825.629149  270515 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751719825.671449  270515 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751719826.108366  270515 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751719826.108713  270515 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751719826.108727  270515 computation_placer.cc:177] computation placer alr

In [6]:
print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [9]:
# print the top‐level Qwen3ForCausalLM structure
print(model)

# or inspect its submodules in detail
for name, module in model.named_modules():
    print(f"{name} -> {module}")

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [ ]:
# model.layers.26.self_attn
model.model.layers[26].self_attn.q_proj

Linear(in_features=1024, out_features=2048, bias=False)

: 

In [23]:
# download dataset
from datasets import load_dataset

DATASET_ID = "iAli61/chempile-education-dedup-dedup"

dataset = load_dataset(DATASET_ID, split="train")

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['url', 'text'],
        num_rows: 28664
    })
})

In [44]:
import tiktoken
import torch
from torch.utils.data import DataLoader, Dataset
COLUMN = "text"  # Column to be used for deduplication

class CHEMPILE_DS_V1(Dataset):
    def __init__(self, dataset, tokenizer, max_length=256, stride=256):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        # add "<|endoftext|>" to the end of each text
        self.texts = " ".join([text + "<|endoftext|>" for text in dataset[COLUMN]][:10])
        self.token_ids = tokenizer.encode(self.texts, allowed_special={"<|endoftext|>"})
        self.input_ids = []
        self.target_ids = []

        # using a sliding window approach to create input-target pairs
        for i in range(0, len(self.token_ids) - self.max_length, stride):
            input_ids = self.token_ids[i:i + self.max_length]
            target_ids = self.token_ids[i + 1:i + self.max_length + 1]
            self.input_ids.append(torch.tensor(input_ids))
            self.target_ids.append(torch.tensor(target_ids))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader(dataset, 
                      tokenizer, 
                      batch_size=8, 
                      max_length=256, 
                      stride=256,
                      drop_last=True,
                      shuffle=True
                      ):

    chempile_ds = CHEMPILE_DS_V1(dataset, tokenizer, max_length=max_length, stride=stride)

    return DataLoader(chempile_ds, 
                      batch_size=batch_size, 
                      drop_last=drop_last,
                      shuffle=shuffle,
                      )


# Example usage
tokenizer = tiktoken.get_encoding("gpt2")
dataloader = create_dataloader(dataset, tokenizer, batch_size=1, max_length=8, stride=8)


dataiter = iter(dataloader)
input_ids, target_ids = next(dataiter)
print(f"Input IDs: {input_ids}")
print(f"Target IDs: {target_ids}")
print(f"Input IDs shape: {input_ids.shape}")
print(f"Target IDs shape: {target_ids.shape}")

print(f"dataset length: {len(dataset)}")
print(f"dataloader length: {len(dataloader)}")


Input IDs: tensor([[43132,    13, 27131,   378,   262,  4894,   286,  9978]])
Target IDs: tensor([[   13, 27131,   378,   262,  4894,   286,  9978,    11]])
Input IDs shape: torch.Size([1, 8])
Target IDs shape: torch.Size([1, 8])
dataset length: 28664
dataloader length: 2339


In [39]:
len(dataloader)

2339

In [27]:
len(dataset)
dataset[:1]

{'url': ['Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Chemical_Thermodynamics_(Supplement_to_Shepherd_et_al.)/04%3A_Fundamental_2_-_Counting_Configurations/4.04%3A_Applying_the_Laws_of_Probability'],
 'text': ['The laws of probability apply to events that are independent. If the result of one trial depends on the result of another trial, we may still be able to use the laws of probability. However, to do so, we must know the nature of the interdependence. If the activity associated with event C precedes the activity associated with event D , the probability of D may depend on whether C occurs. Suppose that the first activity is tossing a coin and that the second activity is drawing a card from a deck; however, the deck we use depends on whether the coin comes up heads or tails. If the coin is heads, we draw a card from an ordinary deck; if the coin is tails, we draw a coin from a deck with the face cards removed. Now we ask about the probability of drawing an ace. If t

## Multihead attention layer

In [7]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [8]:
class MHA(nn.Module):
    def __init__(self, config):
        super(MHA, self).__init__()
        # check if number of heads divides embedding dimension
        if config['emb_dim'] % config['n_heads'] != 0:
            raise ValueError("Embedding dimension must be divisible by number of heads.")
        self.n_heads = config['n_heads']
        self.head_dim = config['emb_dim'] // self.n_heads
        self.config = config
        # dimensions [batch_size, sequence_length, 3 * emb_dim for q, k, v]
        self.qkv = nn.Linear(config['emb_dim'], config['emb_dim'] * 3, bias=config['qkv_bias'])
        self.attn_drop = nn.Dropout(config['drop_rate'])
        self.proj = nn.Linear(config['emb_dim'], config['emb_dim'])
        self.proj_drop = nn.Dropout(config['drop_rate'])

    def forward(self, x):
        B, T, C = x.size()
        # B: batch size, T: sequence length, C: embedding dimension
        # (B, T, 3 * C) -> (B, T, 3, n_heads, head_dim) 
        # where n_heads = config['n_heads'] and head_dim = C // n_heads
        qkv = self.qkv(x).view(B, T, 3, self.config['n_heads'], self.head_dim)
        # (B, T, 3, n_heads, head_dim) -> (3, B, n_heads, T, head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)
        
        use_dropout = 0. if self.training else self.config['drop_rate']
        context_vec = nn.functional.scaled_dot_product_attention(
            q, k, v, 
            attn_mask=None,
            dropout_p=use_dropout, 
            is_causal=True
        )

        # (B, n_heads, T, head_dim) -> (B, T, n_heads * head_dim)
        context_vec = context_vec.transpose(1, 2).contiguous().view(B, T, self.config['emb_dim'])
        # (B, T, emb_dim) -> (B, T, emb_dim)
        context_vec = self.proj(context_vec)

        return context_vec

In [58]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, config):
        super(TransformerBlock, self).__init__()
        self.config = config
        self.mha = MHA(config)
        self.ln1 = nn.LayerNorm(config['emb_dim'])
        self.ffn = nn.Sequential(
            nn.Linear(config['emb_dim'], config['emb_dim'] * 4),
            nn.GELU(),
            nn.Linear(config['emb_dim'] * 4, config['emb_dim'])
        )
        self.ln2 = nn.LayerNorm(config['emb_dim'])
        self.drop = nn.Dropout(config['drop_rate'])

    def forward(self, x):
        # Multi-head attention
        x = x + self.drop(self.mha(self.ln1(x)))
        # Feed-forward network
        x = x + self.drop(self.ffn(self.ln2(x)))

        return x

In [59]:
import time
x = torch.randn(32, 1024, GPT_CONFIG_124M['emb_dim']).cuda()  # Example input tensor
# clean up gpu memory
torch.cuda.empty_cache()
# Measure time for custom Transformer block
transformer_block = TransformerBlock(GPT_CONFIG_124M).cuda()  # Move to GPU if available
start_time = time.time()
output_transformer = transformer_block(x)
transformer_time = time.time() - start_time 

print(f"Transformer block time: {transformer_time:.6f} seconds")

Transformer block time: 1.738435 seconds


In [60]:
class GPT2(torch.nn.Module):

    def __init__(self, config):
        super(GPT2, self).__init__()
        self.config = config
        self.embedding = nn.Embedding(config['vocab_size'], config['emb_dim'])
        self.positional_embedding = nn.Embedding(config['context_length'], config['emb_dim'])
        self.dropout_emb = nn.Dropout(config['drop_rate'])
        # Using nn.Sequential for transformer blocks
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock(config) for _ in range(config['n_layers'])]
        )
        self.ln_f = nn.LayerNorm(config['emb_dim'])
        self.head = nn.Linear(config['emb_dim'], config['vocab_size'], bias=False)

    def forward(self, x):
        B, T = x.size()
        # x: (B, T) where B is batch size and T is sequence length
        # (B, T) -> (B, T, emb_dim)
        x = self.embedding(x) + self.positional_embedding(torch.arange(T, device=x.device))
        x = self.dropout_emb(x)

        # Transformer blocks
        x = self.transformer_blocks(x)

        # Final layer normalization and linear projection
        x = self.ln_f(x)
        logits = self.head(x)

        return logits

In [61]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0).cuda()
print(batch)
print(batch.shape)  # Should be (batch_size, sequence_length)
print(batch.device)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]], device='cuda:0')
torch.Size([2, 4])
cuda:0


In [62]:
tokenizer.encode(txt1)

[6109, 3626, 6100, 345]

In [63]:

# Measure time for custom Transformer block
GPT2_model = GPT2(GPT_CONFIG_124M).cuda()

output_gpt2 = GPT2_model(batch)

print(output_gpt2.shape)  # Should be (batch_size, sequence_length, vocab_size)

output_gpt2


torch.Size([2, 4, 50257])


tensor([[[ 0.1841,  0.9427, -0.6268,  ...,  0.4099,  0.6749, -0.1008],
         [-0.2650,  1.4717, -0.9401,  ...,  0.2010,  0.2585, -0.3980],
         [-0.2078, -0.1370,  0.4248,  ...,  0.2952,  0.2042,  0.2585],
         [-1.4372,  0.2441,  0.2246,  ..., -0.0817, -0.0642,  0.4755]],

        [[ 0.1748,  1.0229, -0.9393,  ..., -0.0309,  1.0036, -0.5670],
         [ 0.4112,  0.9963,  0.5240,  ..., -0.3183,  0.9374,  0.0583],
         [-0.8739, -0.4290, -0.2994,  ..., -0.3820,  0.3080, -0.0327],
         [-1.0862,  0.4183, -0.2846,  ..., -0.9729,  0.0065,  0.4931]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [64]:
total_params = sum(p.numel() for p in GPT2_model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 162,419,712


In [65]:
def text_generation_greedy_V1(model, prompt, tokenizer, context_length=1024, max_new_tokens=50):
    """
    Generate text using greedy decoding.
    
    Args:
        model: The GPT model.
        prompt: The input text prompt.
        tokenizer: The tokenizer for the model.
        context_length: The maximum context length for the model.
        max_new_tokens: The maximum number of new tokens to generate.
        
    Returns:
        Generated text as a string.
    """
    # check if the mode is not gpu, move it to gpu
    if not next(model.parameters()).is_cuda:
        model = model.cuda()
    model.eval()
    input_ids = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).cuda()

    generated_ids = input_ids[:, -(context_length - 1):] # Keep the last context_length - 1 tokens
    generated_ids = generated_ids.cuda()  # Move to GPU if available

    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(generated_ids)
            next_token_logits = outputs[:, -1, :]  # Get logits for the last token
            next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)  # Greedy decoding
            
            generated_ids = torch.cat((generated_ids, next_token_id), dim=1)
            
            if generated_ids.size(1) >= context_length:
                break
    
    generated_text = tokenizer.decode(generated_ids[0].tolist())
    
    return generated_text

In [66]:
prompt = "Every effort moves you"
generated_text = text_generation_greedy_V1(GPT2_model, prompt, tokenizer,
                                           context_length=GPT_CONFIG_124M['context_length'],
                                           max_new_tokens=50)
print(generated_text)

Every effort moves youabl labou knives dungeonsative mother resulting Joyce continuous regexahovez importance tast Needless wrestling revive weLeague Highland Teresa ssh elegance shaped clitor bowel Fleming metast wildProgramlas Introductionverting Phase Reach eviction estimation CommercialIFIED TFentimes differingcus CosponsorsBeautApr Nexeps Submitgang


In [73]:
# number of parameters and flops

from thop import profile

def get_model_parameters_and_flops(model, input_tensor):
    model.eval()
    with torch.no_grad():
        macs, params = profile(model, inputs=(input_tensor,), verbose=False)
    return params, macs*2


model_configs = {
    "gpt-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

input_tensor = torch.randint(0, GPT_CONFIG_124M['vocab_size'], (2, GPT_CONFIG_124M['context_length'])).cuda()

# for model_name, config in model_configs.items():
#     CONFIG = GPT_CONFIG_124M.copy()
#     CONFIG.update(config)  # Update with specific model configuration
#     model = GPT2(CONFIG).cuda()
#     params, flops = get_model_parameters_and_flops(model.bfloat16(), input_tensor)
#     print(f"Number of parameters: {params:.1e} M, FLOPs: {flops:.1e} G for {model_name}")


#     del model  # Delete the model to free up memory
#     torch.cuda.empty_cache()  # Clear GPU memory after each model to avoid OOM errors
torch.cuda.empty_cache()

# Training

### Configurations and Model Initialization


In [74]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPT2(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

### Train/validation data loaders

In [75]:
dataset

Dataset({
    features: ['url', 'text'],
    num_rows: 28664
})

In [76]:
# Train/validation ratio
train_val_split = dataset.train_test_split(test_size=0.1, seed=123)
val_dataset = train_val_split['test']
train_dataset = train_val_split['train']
print(f"Dataset length: {len(dataset)}")
print(f"Training data size: {len(train_val_dataset)}")
print(f"Validation data size: {len(test_dataset)}")


torch.manual_seed(123)

train_loader = create_dataloader(
    train_dataset,
    tokenizer,
    batch_size=2,
    max_length=256,
    stride=256,
    drop_last=True,
    shuffle=True
)

print(f"Number of training batches: {len(train_loader)}")

val_loader = create_dataloader(
    val_dataset,
    tokenizer,
    batch_size=2,
    max_length=256,
    stride=256,
    drop_last=True,
    shuffle=False
)
print(f"Number of validation batches: {len(val_loader)}")

Dataset length: 28664
Training data size: 25797
Validation data size: 2867
Number of training batches: 32
Number of validation batches: 25


In [77]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)
    break

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)
    break

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


### Loss function

In [78]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print(f"Perplexity: {torch.exp(torch.tensor(train_loss)):.2f}")
print("Validation loss:", val_loss)
print(f"Perplexity: {torch.exp(torch.tensor(val_loss)):.2f}")

Using cuda device.
Training loss: 10.993945519129435
Perplexity: 59512.71
Validation loss: 10.981866778749408
Perplexity: 58798.22


### training loop

In [79]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.positional_embedding.weight.shape[0]
    with torch.no_grad():
        generated_text = text_generation_greedy_V1(
            model=model, 
            prompt=start_context,
            tokenizer=tokenizer,
            context_length=context_size,
            max_new_tokens=50   
        )
    print(f"Generated text: {generated_text}")

In [81]:
torch.manual_seed(123)
model = GPT2(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 10.385, Val loss 10.533
Ep 1 (Step 000005): Train loss 8.875, Val loss 9.240
Ep 1 (Step 000010): Train loss 8.062, Val loss 8.619
Ep 1 (Step 000015): Train loss 7.435, Val loss 8.460
Ep 1 (Step 000020): Train loss 7.128, Val loss 8.594
Ep 1 (Step 000025): Train loss 7.021, Val loss 8.636
Ep 1 (Step 000030): Train loss 6.735, Val loss 8.493
Generated text: Every effort moves you the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Ep 2 (Step 000035): Train loss 6.508, Val loss 8.449
Ep 2 (Step 000040): Train loss 6.627, Val loss 8.438
Ep 2 (Step 000045): Train loss 6.564, Val loss 8.637


KeyboardInterrupt: 